In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch.nn.functional as F
from de import DependencyEvolver
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = DependencyEvolver(
    d_model=16,
    dim_feedforward=8,
    nhead=1,
    dropout=0,
    N=5,
    encoder_layers=1,
    decoder_layers=1,
    tok_v=tokenizer.vocab_size,
    rel_v=2,
    pos_v=3
)

In [ ]:
import torch
from torch.optim import AdamW

batch = (
    (8823, 2),
    [
        torch.tensor([[[1, 0, 2, 0, 3]]]),
        torch.tensor([[[-1, -1, 1, -1, -1]]]),
        torch.tensor([[[-1, 2, -1, 2, -1]]]),
        torch.tensor([[[-1, 0, -1, 1, -1]]]),
        torch.tensor([[[-1, 0, -1, 1, -1]]]),
        torch.tensor([[[-1, 2016, -1, 2833, -1]]])
    ]
)

train_loader = [batch for _ in range(1000)]
eval_loader = [batch for _ in range(20)]
optim = AdamW(model.parameters())

In [ ]:
model._train(optim, train_loader, eval_loader, len(train_loader), 20, 1, 1e9, 100)